In [36]:
!pip install djitellopy

In [37]:

!pip install --upgrade djitellopy

# Testes iniciais

In [38]:
from djitellopy import Tello
import time

tello = Tello()

tello.connect()
time.sleep(5)

success = False
attempts = 0
max_attempts = 3


while not success and attempts < max_attempts:
    try:
        tello.takeoff()
        success = True 
    except Exception as e:
        print(f"Attempt {attempts + 1} failed: {e}")
        attempts += 1
        time.sleep(2)  # Espera 2 segundos antes de tentar novamente


tello.move_forward(20)
tello.move_up(30)
tello.flip_back()

tello.land()

[INFO] tello.py - 129 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 438 - Send command: 'command'


KeyError: '192.168.10.1'

In [ ]:
from djitellopy import Tello
import cv2, math, time

tello = Tello()
tello.connect()

battery_level = tello.get_battery()
print(f'Nível:{battery_level}')

tello.streamon()
time.sleep(2)

frame_read = tello.get_frame_read()   
time.sleep(5)

success = False
attempts = 0
max_attempts = 3


while not success and attempts < max_attempts:
    try:
        tello.takeoff()
        success = True 
    except Exception as e:
        print(f"Attempt {attempts + 1} failed: {e}")
        attempts += 1
        time.sleep(2)  # Espera 2 segundos antes de tentar novamente


while True:
    img = frame_read.frame
    cv2.imshow("drone", img)

    key = cv2.waitKey(1) & 0xff
    if key == 27: # ESC        
        cv2.destroyAllWindows()
        tello.streamoff()
        break
    elif key == ord('w'):
        tello.move_forward(30)
    elif key == ord('s'):
        tello.move_back(30)
    elif key == ord('a'):
        tello.flip_back()
    elif key == ord('d'):
        tello.flip_forward()
    elif key == ord('e'):
        tello.rotate_clockwise(30)
    elif key == ord('q'):
        tello.rotate_counter_clockwise(30)
    elif key == ord('r'):
        tello.move_up(30)
    elif key == ord('f'):
        tello.move_down(30)

tello.land()

In [ ]:
import cv2
from djitellopy import Tello

tello = Tello()
tello.connect()

tello.streamon()
frame_read = tello.get_frame_read()

cv2.imwrite("picture.png", frame_read.frame)

tello.streamoff()
# tello.land()

In [ ]:
from djitellopy import Tello
import cv2
import threading

# Conecta ao drone
tello = Tello()
tello.connect()

# Inicia o stream de vídeo
tello.streamon()
frame_read = tello.get_frame_read()


time.sleep(5)

success = False
attempts = 0
max_attempts = 3


while not success and attempts < max_attempts:
    try:
        tello.takeoff()
        success = True 
    except Exception as e:
        print(f"Attempt {attempts + 1} failed: {e}")
        attempts += 1
        time.sleep(2)  # Espera 2 segundos antes de tentar novamente


# Função para exibir o vídeo em uma thread separada
def video_display():
    while True:
        img = frame_read.frame
        cv2.imshow("drone", img)
        if cv2.waitKey(1) & 0xff == 27:  # ESC
            break
    cv2.destroyAllWindows()

# Função para controlar o drone em uma thread separada
def control_drone():
    while True:
        key = cv2.waitKey(1) & 0xff
        if key == ord('w'):
            tello.move_forward(30)
        elif key == ord('s'):
            tello.move_back(30)
        elif key == ord('e'):
            tello.rotate_clockwise(30)
        elif key == ord('q'):
            tello.rotate_counter_clockwise(30)
        elif key == ord('r'):
            tello.move_up(30)
        elif key == ord('f'):
            tello.move_down(30)
        elif key == 27:  # ESC
            tello.land()
            break
        else: 

# Cria e inicia as threads
video_thread = threading.Thread(target=video_display)
control_thread = threading.Thread(target=control_drone)

video_thread.start()
control_thread.start()

# Aguarda as threads terminarem
video_thread.join()
control_thread.join()

# Pousa o drone e encerra
tello.land()


In [8]:
import cv2

# Ajusta os canais das cores captadas pelo drone
imagem = cv2.imread('Captura.png')

(b,g,r) = cv2.split(imagem)


rgb = cv2.merge([r, g, b])

cv2.imshow('rgb', rgb)


cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
from djitellopy import Tello
import time

# Cria a instância do drone
tello = Tello()

# Conecta ao drone
tello.connect()

# Verifica o nível da bateria (opcional)
print(f"Bateria: {tello.get_battery()}%")

# Decola o drone
tello.takeoff()

# Inclina o drone para frente (pitch forward) com velocidade 30
tello.send_rc_control(0, 30, 0, 0)  # (roll, pitch, throttle, yaw)
time.sleep(2)  # Mantém o movimento por 2 segundos

# Para o movimento
tello.send_rc_control(0, 0, 0, 0)
time.sleep(1)

# Inclina para a esquerda (roll) com velocidade 30
tello.send_rc_control(-30, 0, 0, 0)
time.sleep(2)

# Para o movimento
tello.send_rc_control(0, 0, 0, 0)
time.sleep(1)

# Aterrissa o drone
tello.land()

# Fecha a conexão
tello.end()

# Código

In [12]:
from djitellopy import Tello
from ultralytics import YOLO
import cv2
import numpy as np
import cv2
import torch
import time

model = YOLO('YOLO Treinado/runs/detect/train3/weights/best.pt')

tello = Tello()
tello.connect()
        
battery_level = tello.get_battery()

print(f'Nível:{battery_level}')

[INFO] tello.py - 129 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 438 - Send command: 'command'
[INFO] tello.py - 462 - Response command: 'ok'


Nível:93


In [14]:
def idYolo(img, height, width):
    
    # Variáveis dos retângulos de referência
    bwRect1 = int(0.3*width)
    ewRect1 = int(0.7*width)
    bhRect2 = int(0.3*height)
    ehRect2 = int(0.7*height)
    
    result = model(img)
    img = result[0].plot()


    # Desenho dos retângulos na tela - referência    
    cv2.rectangle(img, (bwRect1, 0), (ewRect1, height), (0,255,0), 2)    
    cv2.rectangle(img, (0, bhRect2), (width, ehRect2), (0,255,0), 2) 
    

    if not len(result[0].boxes.xywh):
        print('Bolas não detectadas')
    
    else:
        # Quantidade de bounding boxes detectadas == Quantidade de objetos detectados
        size = len(result[0].boxes.xywh)
        
        boxesxywh = result[0].boxes.xywh
        print(result[0].boxes.conf)    
        
        # Confiabilidade da previsão
        conf = result[0].boxes.conf
        
        # Bola com maior confiabilidade 
        i = torch.argmax(conf)       
        
        # Coordenadas da bola com maior confiabilidade
        x,y,w,h = boxesxywh[i]
        cv2.putText(img, str(h.item()), (10,50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,255,0), 2)
        
        # if h.item() < 50: ##COLOCAR PARAMETRO ADEQUADO DE DISTANCIA (D_mínima)
         #     tello.send_rc_control(0,0,0,0)
        #     cv2.putText(img, 'bolinha identificada - distancia minima atingida', (10,30), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,255,0), 2)
        # else:
        if x > bwRect1 and x < ewRect1 and y > bhRect2 and y < ehRect2:
                cv2.putText(img, 'centro', (10,30), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,255,0), 2)
                tello.move_forward(20)
        elif x < bwRect1 and y > bhRect2 and y < ehRect2:
                cv2.putText(img, 'esquerda - centro', (10,30), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,255,0), 2)
                tello.rotate_counter_clockwise(15)
        elif x > ewRect1 and y > bhRect2 and y < ehRect2:
                cv2.putText(img, 'direita - centro', (10,30), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,255,0), 2)
                tello.rotate_clockwise(15)
        elif x > bwRect1 and x < ewRect1 and y < bhRect2:
                cv2.putText(img, 'em cima - centro', (10,30), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,255,0), 2)
                tello.move_up(20)
        elif x > bwRect1 and x < ewRect1 and y > ehRect2:
                cv2.putText(img, 'embaixo - centro', (10,30), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,255,0), 2)
                tello.move_down(20)
        elif x < bwRect1 and y < bhRect2: 
                cv2.putText(img, 'em cima - esquerda', (10,30), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,255,0), 2)
                tello.move_up(20)
                tello.rotate_counter_clockwise(15)         
        elif x < bwRect1 and y > ehRect2: 
                cv2.putText(img, 'embaixo - esquerda', (10,30), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,255,0), 2)
                tello.move_up()
                tello.rotate_counter_clockwise(15)
        elif x > ewRect1 and y < bhRect2: 
                cv2.putText(img, 'em cima - direita', (10,30), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,255,0), 2)
                tello.move_up(20)
                tello.rotate_clockwise(15)            
        elif x > ewRect1 and y > ehRect2: 
                cv2.putText(img, 'embaixo - direita', (10,30), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,255,0), 2)
                tello.move_down(20)
                tello.rotate_clockwise(15)
                

        
        # Centro de massa da bolinha
        cv2.rectangle(img, (int(x),int(y)), (int(x),int(y)), (0,255,255),3)
        
        # Vetor de direção em relação ao centro da imagem
        cv2.arrowedLine(img, (int(width/2),int(height/2)), (int(x),int(y)), (255,0,0), 2)
    
    return img            
        

error while decoding MB 34 31, bytestream -9


In [15]:
# Inicia o stream de vídeo
tello.streamon()
time.sleep(2)

tello.takeoff()
frame_read = tello.get_frame_read()

while True:
    tello.send_rc_control(0,0,0,0)

    (r,g,b) = cv2.split(frame_read.frame)
    img = cv2.merge([b,g,r])
    # width, height, _ = img.shape
    resized_img = cv2.resize(img, (480, 369))     
    img = idYolo(resized_img, 369, 480)
    
    # img = cv2.resize(img,(640,480))
                
            
    cv2.imshow("drone", img)
    if cv2.waitKey(1) & 0xff == 27:  # ESC
        break
    
cv2.destroyAllWindows()
tello.streamoff()
tello.land()

[INFO] tello.py - 438 - Send command: 'streamon'
[INFO] tello.py - 462 - Response streamon: 'ok'
left block unavailable for requested intra mode
error while decoding MB 0 24, bytestream 336
[INFO] tello.py - 438 - Send command: 'takeoff'
left block unavailable for requested intra mode
error while decoding MB 0 25, bytestream 813
left block unavailable for requested intra mode
error while decoding MB 0 6, bytestream 4626
left block unavailable for requested intra4x4 mode -1
error while decoding MB 0 17, bytestream 3728
left block unavailable for requested intra mode
error while decoding MB 0 14, bytestream 3916
error while decoding MB 30 35, bytestream -6
[INFO] tello.py - 462 - Response takeoff: 'ok'
bind failed: Error number -10048 occurred


OSError: [Errno 10038] Error number -10038 occurred: 'udp://@0.0.0.0:11111'; last error log: [udp] bind failed: Error number -10048 occurred

error while decoding MB 10 40, bytestream -6
error while decoding MB 6 41, bytestream -8
left block unavailable for requested intra4x4 mode -1
error while decoding MB 0 12, bytestream 3573
left block unavailable for requested intra mode
error while decoding MB 0 14, bytestream 2557
error while decoding MB 10 39, bytestream -10


In [ ]:
cv2.destroyAllWindows()
tello.land()
tello.streamoff()

left block unavailable for requested intra4x4 mode -1
error while decoding MB 0 17, bytestream 3760
error while decoding MB 3 31, bytestream -6
